<a href="https://colab.research.google.com/github/Kativic/Projeto1/blob/main/Exer_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalação de bibliotecas

In [ ]:
pip install gcsfs

In [ ]:
pip install pandera

##Declaração das LIBS

In [ ]:
import pandas as pd
import os
import numpy as np
from google.cloud import storage
import pandera as pa

##Config . do pandas

In [ ]:
pd.set_option('display.max_columns',100)

##Conector para Cloud Storage(bucket)

In [ ]:
#CONFIGURANDO DA CHAVE DE SEGURANCA (Enviada com o projeto)

serviceAccount = '/content/ad1-soulcode-590cc7ea2211.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
client = storage.Client()
bucket = client.get_bucket('katiasantos01')
bucket.blob('amsterdam_weekdays (1).csv')
path = 'gs://katiasantos01/brutos /amsterdam_weekdays (1).csv'

##Extração dos dados(Extract)

In [ ]:
df = pd.read_csv(path,sep=';',encoding='ISO-8859-1',parse_dates=['ocorrencia_dia'],dayfirst=True)

##Pré analise

In [ ]:
#Visualização do arquivo(dados)
df

In [ ]:
#Vis . do df começando pelo cabeçalho
df.head(3)

In [ ]:
#Vis . do df começando pelo final 
df.tail(1)

In [ ]:
#Tipo de dados das colunas
df.dtypes

In [ ]:
#Tam. do DF
df.shape

(6769, 22)

In [ ]:
#Info detalhada do DF
df.info()

In [ ]:
#Verifica se dados são unicos na coluna
df.codigo_ocorrencia.is_unique

True

In [ ]:
#Setar novo rótulo do DF
df.set_index('codigo_ocorrencia',inplace=True)

In [ ]:
#resetar o Rotulo(index) do DF
df.reset_index(inplace=True)

##Tranformações (Limpeza, conversão , etc)

In [ ]:
#Backup local(M.RAM) do df
dfback = df.copy()

In [ ]:
# Drop de colunas identicas ( codigo_ocorrencia1 , codigo_ocorrencia2,codigo_ocorrencia3 , codigo_ocorrencia4)
df.drop(['codigo_ocorrencia1','codigo_ocorrencia2','codigo_ocorrencia3','codigo_ocorrencia4'],axis=1,inplace=True)
# Drop de colunas com problemas e/ou não utilizadas na analise
df.drop(['ocorrencia_latitude','ocorrencia_longitude','investigacao_aeronave_liberada','investigacao_status','divulgacao_relatorio_numero','divulgacao_relatorio_publicado','divulgacao_dia_publicacao'],axis=1,inplace=True)

In [ ]:
df.drop(['data','horario'],axis=1,inplace=True)

In [ ]:
#Renomeando colunas
df.rename(columns={'ocorrencia_classificacao':'classificacao','ocorrencia_cidade':'cidade','ocorrencia_uf':'uf','ocorrencia_pais':'pais'},inplace=True)
df.rename(columns={'ocorrencia_aerodromo':'aerodromo','ocorrencia_dia':'data','ocorrencia_hora':'horario','total_aeronaves_envolvidas':'n_aeronaves','ocorrencia_saida_pista':'saida_pista'},inplace=True)

In [ ]:
#Visualiza valores unicos na coulna
pd.unique(df['aerodromo'])

Inconsistencias encontradas
colunas aerodromo , uf

```
('***', '**NI')
```



In [ ]:
df.replace(['***','**NI'],pd.NA,inplace=True)

In [ ]:
df.replace(pd.NA,np.NaN,inplace=True)

In [ ]:
#Converter coluna data para string
df['data'] = df['data'].astype(str)

In [ ]:
#Criar coluna periodo (data + horario)
df['periodo'] = pd.to_datetime(df['data']+' '+df['horario'])

In [ ]:
#Verificar dados Nulos , ausentes , etc
df.isna().sum()

In [ ]:
df.count()

##Criação do Schema de validação de dados

In [ ]:
df.dtypes

In [ ]:
#Criação do schema pandera

schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia':pa.Column(pa.Int),
        'classificacao':pa.Column(pa.String),
        'cidade':pa.Column(pa.String),
        'uf':pa.Column(pa.String,pa.Check.str_length(2,2),nullable=True),
        'pais':pa.Column(pa.String),
        'aerodromo':pa.Column(pa.String,pa.Check.str_length(4,4),nullable=True),
        'data':pa.Column(pa.DateTime),
        'horario':pa.Column(pa.String,pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$'),nullable=True),
        'total_recomendacoes':pa.Column(pa.Int),
        'n_aeronaves':pa.Column(pa.Int),
        'saida_pista':pa.Column(pa.String,pa.Check.str_length(3,3)),
    }
)

schema2 = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia':pa.Column(pa.Int),
        'classificacao':pa.Column(pa.String),
        'cidade':pa.Column(pa.String),
        'uf':pa.Column(pa.String,pa.Check.str_length(2,2),nullable=True),
        'pais':pa.Column(pa.String),
        'aerodromo':pa.Column(pa.String,pa.Check.str_length(4,4),nullable=True),
        'total_recomendacoes':pa.Column(pa.Int),
        'n_aeronaves':pa.Column(pa.Int),
        'saida_pista':pa.Column(pa.String,pa.Check.str_length(3,3)),
        'periodo':pa.Column(pa.DateTime,nullable=True),
    }
)

In [ ]:
df.loc[0,'aerodromo'] = 'SBSP'

In [ ]:
schema2.validate(df)

##Busca(query)

In [ ]:
#alterar um valor 
df.loc[0,'aerodromo'] = 'SBSP'

In [ ]:
#busca avançada
df.loc[[0,1000,2000],['classificacao','uf']]

In [ ]:
df.loc[df.classificacao == 'INCIDENTE GRAVE',['classificacao']] = 'YURI'

In [ ]:
df.loc[df.classificacao == 'YURI',['classificacao']] = 'INCIDENTE GRAVE'

In [ ]:
df['backaero'] = df.aerodromo

In [ ]:
df.loc[0,'n_aeronaves'] = '1'

In [ ]:
#conversão do tipo das colunas
df['n_aeronaves'] = df['n_aeronaves'].astype('int')

##Filtros

In [ ]:
df.isna().sum()

In [ ]:
filtronulo = df.uf.isna()
df.loc[filtronulo]

In [ ]:
filtroincidente = df.classificacao =='INCIDENTE'
#dfacidente = df.loc[filtroacidente]

In [ ]:
#filtro com mais detalhes
ftcidade = df.cidade == 'SÃO PAULO'
ftpista = df.saida_pista  == 'SIM'
ft2021 = df.data.dt.year == 2021
ftuf = df.uf == 'SP'

In [ ]:
dfsp = df.loc[ftpista & ft2021 & ftuf]

In [ ]:
ftabril = df.data.dt.month == 4
ftquinz = (df.data.dt.day > 0) & (df.data.dt.day <16)
df.loc[ftabril & ftquinz]

In [ ]:
# Filtro para palavras terminadas em RIO
filtroletras = df.cidade.str[-3:] == 'RIO'
df.loc[filtroletras]

In [ ]:
#Filtro para letras dentro do valor
filtrointerno = df.cidade.str.contains('MA')

In [ ]:
df.loc[filtrointerno]

##GroupBY ( Agrupamentos)

In [ ]:
df.groupby(['classificacao']).size().sort_values(ascending=False)

In [ ]:
df.groupby(['aerodromo'],dropna=False).size().sort_values(ascending=False)

In [ ]:
dfacidente.groupby(['aerodromo'],dropna=False).size().sort_values(ascending=False)

In [ ]:
#filtro sul e incidentes
filtrosul = df.uf.isin(['RS','PR','SC'])
dfsul = df.loc[filtroincidente & filtrosul]

In [ ]:
dfsul.groupby(['uf']).size().sort_values(ascending=False)

In [ ]:
dfsp.groupby(['classificacao']).size().sort_values(ascending=False)

##Plotagem

In [ ]:
df.groupby(['uf'],dropna=False).size().sort_values(ascending=False).plot.bar(figsize=(12,8),xlabel='ESTADO',ylabel='N. Ocorrencias')

In [ ]:
df.groupby('saida_pista',dropna=False).size().sort_values(ascending=False).plot.pie()

##LOAD (Carregamento) salvar o df.

In [ ]:
#Utilizar se for o rotulo padrão 
df.to_csv('cenipa_tratado_ok.csv',index=False)

In [ ]:
#Carregar direto no bucket
df.to_csv('gs://igorgondim01/tratados/cenipa_tratado.csv',index=False)